## Assignment 2: Probabilistic Models and Vector Space Applications

### Assignment Overview

This assignment builds on fundamental text processing techniques to explore probabilistic language modeling, text classification, and the practical application of vector space models for information retrieval. You will implement a simple n-gram language model, build a complete text classification pipeline, develop a search engine, and analyze the core components of sequence models.

You are required to complete five coding-related tasks. For each task, you will be working with a specified dataset or corpus. Please submit your solutions in this single Jupyter Notebook (`.ipynb`) file, clearly marking each task. Ensure your code is well-commented and your findings are explained in markdown cells where requested.

### Task 1: Implementing a Bigram Language Model with Laplace Smoothing (20 Marks)

**Objective:** To understand the fundamentals of n-gram language models, including probability calculation, smoothing, and evaluation with perplexity.

**Description:** You will implement a Bigram language model from scratch. Your model will be trained on a small corpus and will use Add-One (Laplace) smoothing to handle unseen n-grams.

**Your task is to:**

1.  **Implement a training function `train_bigram_model(corpus)`:**
    * The corpus will be a list of sentences, where each sentence is a list of tokens.
    * The function should count all unigrams and bigrams in the corpus.
    * It should return the unigram counts, bigram counts, and the vocabulary size (V).

2.  **Implement a probability function `calculate_bigram_prob(prev_word, word, unigram_counts, bigram_counts, V)`:**
    * This function should calculate the smoothed probability of a `word` given the `prev_word` using the formula for Laplace (Add-One) smoothing: $P(w_i | w_{i-1}) = \frac{C(w_{i-1}, w_i) + 1}{C(w_{i-1}) + V}$.

3.  **Implement a perplexity calculation function `calculate_perplexity(sentence, ...)`:**
    * This function should take a test sentence and your trained model components as input.
    * It should calculate the perplexity of the sentence using the formula: $PP(W) = P(w_1, w_2, ..., w_N)^{-1/N}$. Remember to handle the start of the sentence appropriately (e.g., by assuming a start token `<S>`).

4.  **Train and Evaluate:**
    * Train your model on the provided `train_corpus`.
    * Calculate and print the perplexity of your model on the `test_sentence`.

**Corpus:**

```python
# Sample corpus for training and testing
train_corpus = [["<S>", "i", "am", "sam", "</S>"], ["<S>", "sam", "i", "am", "</S>"], ["<S>", "i", "do", "not", "like", "green", "eggs", "and", "ham", "</S>"]]
test_sentence = ["<S>", "i", "like", "green", "ham", "</S>"]
```

In [ ]:
# Your code for Task 1 here
import numpy as np
from collections import Counter, defaultdict

# Provided Corpus
train_corpus = [["<S>", "i", "am", "sam", "</S>"], ["<S>", "sam", "i", "am", "</S>"], ["<S>", "i", "do", "not", "like", "green", "eggs", "and", "ham", "</S>"]]
test_sentence = ["<S>", "i", "like", "green", "ham", "</S>"]

def train_bigram_model(corpus):
    """
    ?? Bigram ????
    
    ??:
        corpus: ????????????? token ??
    
    ??:
        unigram_counts: unigram ????
        bigram_counts: bigram ????
        V: ?????
    """
    unigram_counts = Counter()
    bigram_counts = defaultdict(Counter)
    
    # ???? unigram ? bigram
    for sentence in corpus:
        # ?? unigram
        for token in sentence:
            unigram_counts[token] += 1
        
        # ?? bigram
        for i in range(len(sentence) - 1):
            prev_word = sentence[i]
            word = sentence[i + 1]
            bigram_counts[prev_word][word] += 1
    
    # ????????????? token?
    V = len(unigram_counts)
    
    return unigram_counts, bigram_counts, V

def calculate_bigram_prob(prev_word, word, unigram_counts, bigram_counts, V):
    """
    ???? Laplace ??? bigram ??
    
    ??: P(w_i | w_{i-1}) = (C(w_{i-1}, w_i) + 1) / (C(w_{i-1}) + V)
    
    ??:
        prev_word: ????
        word: ???
        unigram_counts: unigram ????
        bigram_counts: bigram ????
        V: ?????
    
    ??:
        ???????
    """
    # ?? bigram ?????????? 0
    bigram_count = bigram_counts.get(prev_word, Counter()).get(word, 0)
    
    # ??????? unigram ?????????? 0
    prev_word_count = unigram_counts.get(prev_word, 0)
    
    # ?? Laplace ????
    prob = (bigram_count + 1) / (prev_word_count + V)
    
    return prob

def calculate_perplexity(sentence, unigram_counts, bigram_counts, V):
    """
    ????????
    
    ??: PP(W) = P(w_1, w_2, ..., w_N)^{-1/N}
    ?? P(w_1, w_2, ..., w_N) = ? P(w_i | w_{i-1})
    
    ??:
        sentence: ?????token ??
        unigram_counts: unigram ????
        bigram_counts: bigram ????
        V: ?????
    
    ??:
        ????
    """
    # ???? bigram ?????
    log_prob_sum = 0.0
    
    # ???????? bigram
    for i in range(len(sentence) - 1):
        prev_word = sentence[i]
        word = sentence[i + 1]
        
        # ????
        prob = calculate_bigram_prob(prev_word, word, unigram_counts, bigram_counts, V)
        
        # ??????????
        log_prob_sum += np.log(prob)
    
    # ??????????????? </S>???????? bigram ???
    N = len(sentence) - 1
    
    # ?????: PP(W) = exp(-log_prob_sum / N)
    perplexity = np.exp(-log_prob_sum / N)
    
    return perplexity

# Train the model
unigram_counts, bigram_counts, V = train_bigram_model(train_corpus)

# Calculate and print perplexity
perplexity = calculate_perplexity(test_sentence, unigram_counts, bigram_counts, V)
print(f"Perplexity of the test sentence: {perplexity:.2f}")


### Task 2: Text Classification with TF-IDF and Naive Bayes (20 Marks)

**Objective:** To build a complete text classification pipeline using TF-IDF feature extraction and a Multinomial Naive Bayes classifier.

**Description:** You will use `scikit-learn` to classify SMS messages as either "spam" or "ham" (not spam). This task integrates vector space representation with a classic probabilistic model.

**Your task is to:**

1.  Load the SMS Spam Collection dataset.
2.  Split the dataset into an 80% training set and a 20% testing set.
3.  Create a text processing pipeline using `sklearn.pipeline.Pipeline` that consists of two steps:
    * `TfidfVectorizer`: To convert text messages into TF-IDF vectors. Use the default parameters.
    * `MultinomialNB`: The Multinomial Naive Bayes classifier.
4.  Train the pipeline on the training data.
5.  Evaluate the trained model on the testing data. Print the following:
    * The accuracy of the model.
    * A full classification report (including precision, recall, and F1-score for each class) using `sklearn.metrics.classification_report`.
6.  Use the trained pipeline to predict the class of two new messages: `"Congratulations! You've won a $1,000 gift card. Go to http://example.com to claim now."` and `"Hi mom, I'll be home for dinner tonight."`

**Dataset:**

* **SMS Spam Collection Dataset:** A public set of SMS labeled messages.
* **Access:** Download from the UCI Machine Learning Repository: [SMS Spam Collection](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You will need the `SMSSpamCollection` file.

In [ ]:
# Your code for Task 2 here
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Load the dataset, e.g, as follows. But you may modify it.
# df = pd.read_csv('path_to_your_dataset/SMSSpamCollection', sep='\t', header=None, names=['label', 'message'])

# Split data

# Create and train the pipeline

# Evaluate the model

# Predict on new messages


### Task 3: Building a Simple Information Retrieval System (20 Marks)

**Objective:** To apply TF-IDF and Cosine Similarity to build a basic document retrieval system that ranks documents based on their relevance to a query.

**Description:** You will create a system that takes a text query and returns the most relevant documents from a small corpus. This is the core principle behind search engines.

**Your task is to:**

1.  Use the provided `document_corpus`.
2.  Create a `TfidfVectorizer` and fit it on the corpus to learn the vocabulary and IDF weights.
3.  Transform the corpus into a TF-IDF document-term matrix.
4.  Write a function `rank_documents(query, vectorizer, doc_term_matrix, top_n=3)` that:
    * Takes a `query` string, the fitted `vectorizer`, the document-term `matrix`, and an optional `top_n` integer.
    * Transforms the input query into a TF-IDF vector using the *same* vectorizer.
    * Calculates the cosine similarity between the query vector and all document vectors in the matrix.
    * Returns the indices and content of the `top_n` most similar documents.
5.  Demonstrate your system by running it with the query `"deep learning models for vision"` and printing the ranked results.

**Dataset:**

```python
# A small corpus of document abstracts
document_corpus = [
    "The field of machine learning has seen rapid growth in recent years, especially in deep learning.",
    "Natural language processing allows machines to understand and respond to human text.",
    "Computer vision focuses on enabling computers to see and interpret the visual world.",
    "Deep learning models like convolutional neural networks are powerful for computer vision tasks.",
    "Recurrent neural networks are often used for sequential data in natural language processing."
    ...
]
```

In [ ]:
# Your code for Task 3 here
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

document_corpus = [
    "The field of machine learning has seen rapid growth in recent years, especially in deep learning.",
    "Natural language processing allows machines to understand and respond to human text.",
    "Computer vision focuses on enabling computers to see and interpret the visual world.",
    "Deep learning models like convolutional neural networks are powerful for computer vision tasks.",
    "Recurrent neural networks are often used for sequential data in natural language processing.",
    "The advances in reinforcement learning have led to breakthroughs in game playing and robotics.",
    "Transfer learning enables models trained on large datasets to be adapted for new tasks with limited data.",
    "Unsupervised learning techniques can discover hidden patterns in data without labeled examples.",
    "Optimization algorithms such as stochastic gradient descent are crucial for training neural networks.",
    "Attention mechanisms have improved the performance of natural language translation and image captioning.",
    "Generative adversarial networks create realistic images and are used for data augmentation.",
    "Feature engineering and selection are important steps in classical machine learning pipelines.",
    "Object detection is a key task in computer vision that involves locating instances within images.",
    "The combination of convolutional and recurrent networks is used for video classification tasks.",
    "Zero-shot learning allows models to recognize objects and concepts they have not seen during training.",
    "Natural language generation is used for creating text summaries and chatbot responses.",
    "Graph neural networks leverage graph structures for tasks such as social network analysis and chemistry.",
    "Hyperparameter tuning can significantly improve the accuracy of deep learning models.",
    "Cross-modal learning involves integrating information from multiple data sources such as text and images.",
    "Evaluating model performance requires a good choice of metrics such as F1-score and RMSE."
]

# Create and fit the vectorizer

# Transform the corpus to DTM

def rank_documents(query, vectorizer, doc_term_matrix, top_n=3):
    # Implement the ranking function
    pass

# Demonstrate with a sample query
query = "deep learning models for vision"
# ranked_docs = rank_documents(query, vectorizer, doc_term_matrix)
# print(f"Top {len(ranked_docs)} documents for the query: '{query}'\n")
# for idx, doc in ranked_docs:
#     print(f"Rank {idx+1}: {doc}")


### Task 4: Implementing Viterbi for HMM POS Tagging (20 Marks)

**Objective:** Implement a simple Hidden Markov Model (HMM) POS tagger via the Viterbi algorithm.

**Description:** Implement Viterbi decoding for a small HMM and apply it to two sentences with the ambiguous word "book". Then briefly discuss why HMMs work for POS tagging and a limitation of the Markov assumption.

**Your task is to:**

1. Define two sentences:
    * `sentence1 = "The book is on the table."`
    * `sentence2 = "I want to book a flight."`
2. Implement Viterbi in log-space for a small tag set (e.g., `{DET, NOUN, VERB, PRT}`). Use the example initial (?), transition (A), and emission (B) probabilities in the parameters block below, or define your own consistent matrices and document them.
3. Run your decoder on both sentences and print the predicted tag sequence and total log-probability.
4. In a markdown cell, explain:
    * **a)** How transition and emission probabilities lead to different tags for "book" in the two sentences.
    * **b)** One sentence where the first-order Markov assumption is limiting, and why.

**Parameters:** Use the matrices shown in the section ?Viterbi decoding for a simple HMM (Task 4)? below.

#### Viterbi decoding for a simple HMM (Task 4)

We illustrate HMM POS tagging with a small tag set `T = {DET, NOUN, VERB, PRT}` and vocabulary `V = {the, a, book, table, flight, is, want, to, on, i}`. The HMM comprises initial probabilities ?, tag-to-tag transitions A, and tag-to-word emissions B.

Example parameters (each row sums to 1):

- Initial ?:
  - P(DET)=0.50, P(NOUN)=0.20, P(VERB)=0.20, P(PRT)=0.10

- Transition A (rows: from-tag, cols: to-tag) in order [DET, NOUN, VERB, PRT]:

```text
from\to   DET    NOUN   VERB   PRT
DET      0.05   0.75   0.15   0.05
NOUN     0.05   0.10   0.75   0.10
VERB     0.10   0.35   0.40   0.15
PRT      0.05   0.10   0.75   0.10
```

- Emission B:
  - DET: the(0.80), a(0.20)
  - NOUN: book(0.45), table(0.25), flight(0.20), i(0.05), on(0.05)
  - VERB: is(0.40), want(0.35), book(0.20), to(0.03), on(0.02)
  - PRT: to(0.70), on(0.30)

Viterbi recurrence in log-space to avoid underflow:

- Initialization: `V[tag, 0] = log ?[tag] + log B[tag, x0]`
- Recurrence: `V[tag, i] = log B[tag, xi] + max_prev ( V[prev, i-1] + log A[prev->tag] )`
- Backtrace from the best final tag.

We will decode the most likely tag sequence for the two Task 4 sentences using these parameters.


In [ ]:
# Task 4: Implement Viterbi for a simple POS HMM (skeleton)
import math
from typing import List, Dict, Tuple

# Define your tag set
TAGS = ["DET", "NOUN", "VERB", "PRT"]

# Define HMM parameters (fill using the given parameters)
pi: Dict[str, float] = {
    # "DET": ..., "NOUN": ..., "VERB": ..., "PRT": ...
}

A: Dict[str, Dict[str, float]] = {
    # "DET": {"DET": ..., "NOUN": ..., "VERB": ..., "PRT": ...},
    # "NOUN": {"DET": ..., "NOUN": ..., "VERB": ..., "PRT": ...},
    # "VERB": {"DET": ..., "NOUN": ..., "VERB": ..., "PRT": ...},
    # "PRT": {"DET": ..., "NOUN": ..., "VERB": ..., "PRT": ...},
}

B: Dict[str, Dict[str, float]] = {
    # "DET": {"the": ..., "a": ...},
    # "NOUN": {"book": ..., "table": ..., "flight": ..., "i": ..., "on": ...},
    # "VERB": {"is": ..., "want": ..., "book": ..., "to": ..., "on": ...},
    # "PRT": {"to": ..., "on": ...},
}

UNK = 1e-8

# Return log-probability for emitting 'word' from 'tag'. Use UNK for unseen words.
def emission_logprob(tag: str, word: str) -> float:
    # TODO: implement emission in log-space
    pass

# Implement Viterbi in log-space
# 1) initialize  2) dynamic programming with backpointers  3) termination + backtrace
def viterbi(tokens: List[str]) -> Tuple[List[str], float]:
    # TODO: implement Viterbi
    pass

# Prepare the two sentences (lowercased tokens)
sentence1 = ["the", "book", "is", "on", "the", "table"]
sentence2 = ["i", "want", "to", "book", "a", "flight"]

# Run your decoder and print outputs
# tags1, logp1 = viterbi(sentence1)
# print(list(zip(sentence1, tags1)), " | logP=", round(logp1, 3))
# tags2, logp2 = viterbi(sentence2)
# print(list(zip(sentence2, tags2)), " | logP=", round(logp2, 3))


**Analysis for Task 4**

* How transition and emission probabilities lead to different tags for "book" in the two sentences.


* One sentence where the first-order Markov assumption is limiting, and why.

Your analysis goes here    

### Task 5: Comparing Cosine Similarity and Euclidean Distance (20 Marks)

**Objective:** To empirically demonstrate the difference between angle-based (Cosine) and magnitude-based (Euclidean) similarity measures in a vector space.

**Description:** The choice of similarity metric is crucial. This task highlights how document length affects each metric and why Cosine Similarity is often preferred for text-based topic similarity.

**Your task is to:**

1.  Define three simple documents:
    * `doc_A = "The cat sat on the mat."`
    * `doc_B = "The cat sat on the mat. The dog chased the cat."` (Longer, but on the same topic)
    * `doc_C = "The rocket launched into space."` (Different topic)
2.  Use `sklearn.feature_extraction.text.CountVectorizer` to transform these three documents into count vectors.
3.  Calculate the **Cosine Similarity** between all unique pairs of documents (A-B, A-C, B-C).
4.  Calculate the **Euclidean Distance** between all unique pairs of documents.
5.  Display your results clearly, for instance, in a Pandas DataFrame.
6.  **In a markdown cell, analyze your results:**
    * Explain why the Cosine Similarity between `doc_A` and `doc_B` is high, while their Euclidean Distance is relatively large.
    * Which metric (Cosine Similarity or Euclidean Distance) do your results suggest is better for identifying documents with similar topics, regardless of their length? Justify your answer based on your calculations.

In [ ]:
# Your code for Task 5 here
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import pandas as pd

# Define documents
doc_A = "The cat sat on the mat."
doc_B = "The cat sat on the mat. The dog chased the cat."
doc_C = "The rocket launched into space."
corpus = [doc_A, doc_B, doc_C]

# Vectorize the documents

# Calculate similarities and distances

# Display results in a DataFrame


#### **Analysis for Task 5**

* Explain why the Cosine Similarity between `doc_A` and `doc_B` is high, while their Euclidean Distance is relatively large.

* Which metric (Cosine Similarity or Euclidean Distance) do your results suggest is better for identifying documents with similar topics, regardless of their length? Justify your answer based on your calculations.

*Your analysis goes here.*